In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Crawl data Viblo**

In [2]:
import requests
import pandas as pd
import time

# =====================================================
# CONFIG
# =====================================================
BASE_LIST_URL = "https://viblo.asia/api/posts/newest?page="
BASE_DETAIL_URL = "https://viblo.asia/api/posts/"
PAGES = 350          # ~350 page * 30 bài ≈ 10,500 bài
LIMIT = 30
SLEEP_TIME = 0.5
OUTPUT_CSV = "viblo_posts_with_tags.csv"

# =====================================================
# 1. CRAWL DANH SÁCH BÀI VIẾT (KHÔNG TAG)
# =====================================================
def crawl_post_list(pages, limit):
    posts = []
    for page in range(1, pages + 1):
        url = f"{BASE_LIST_URL}{page}&limit={limit}"
        r = requests.get(url)
        if r.status_code == 200:
            print(f"[LIST] Page {page} OK")
            posts.extend(r.json()["data"])
        else:
            print(f"[LIST] Page {page} FAIL")
        time.sleep(SLEEP_TIME)
    return posts

# =====================================================
# 2. CRAWL CHI TIẾT TỪNG BÀI (CÓ TAG = NHÃN)
# =====================================================
def crawl_post_detail(slug):
    url = BASE_DETAIL_URL + slug
    r = requests.get(url)
    if r.status_code != 200:
        return None

    post = r.json().get("post", {}).get("data", {})
    if not post:
        return None

    tags = post.get("tags", {}).get("data", [])
    tag_names = [t["name"] for t in tags]

    return {
        "id": post.get("id"),
        "title": post.get("title"),
        "content": post.get("contents"),
        "tags": "|".join(tag_names)   # MULTI-LABEL
    }

# =====================================================
# 3. PIPELINE CHÍNH
# =====================================================
def crawl_viblo_with_tags(pages, limit):
    post_list = crawl_post_list(pages, limit)
    results = []

    for idx, post in enumerate(post_list):
        slug = post.get("slug")
        if not slug:
            continue

        detail = crawl_post_detail(slug)
        if detail:
            results.append(detail)
            print(f"[DETAIL] {idx+1}/{len(post_list)} OK")
        else:
            print(f"[DETAIL] {idx+1}/{len(post_list)} FAIL")

        time.sleep(SLEEP_TIME)

    return results

# =====================================================
# 4. CHẠY & LƯU CSV
# =====================================================
if __name__ == "__main__":
    data = crawl_viblo_with_tags(PAGES, LIMIT)
    df = pd.DataFrame(data)
    df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
    print(f"\nDONE ✅ Saved {len(df)} rows to {OUTPUT_CSV}")


[LIST] Page 1 OK
[LIST] Page 2 OK
[LIST] Page 3 OK
[LIST] Page 4 OK
[LIST] Page 5 OK
[LIST] Page 6 OK
[LIST] Page 7 OK
[LIST] Page 8 OK
[LIST] Page 9 OK
[LIST] Page 10 OK
[LIST] Page 11 OK
[LIST] Page 12 OK
[LIST] Page 13 OK
[LIST] Page 14 OK
[LIST] Page 15 OK
[LIST] Page 16 OK
[LIST] Page 17 OK
[LIST] Page 18 OK
[LIST] Page 19 OK
[LIST] Page 20 OK
[LIST] Page 21 OK
[LIST] Page 22 OK
[LIST] Page 23 OK
[LIST] Page 24 OK
[LIST] Page 25 OK
[LIST] Page 26 OK
[LIST] Page 27 OK
[LIST] Page 28 OK
[LIST] Page 29 OK
[LIST] Page 30 OK
[LIST] Page 31 OK
[LIST] Page 32 OK
[LIST] Page 33 OK
[LIST] Page 34 OK
[LIST] Page 35 OK
[LIST] Page 36 OK
[LIST] Page 37 OK
[LIST] Page 38 OK
[LIST] Page 39 OK
[LIST] Page 40 OK
[LIST] Page 41 OK
[LIST] Page 42 OK
[LIST] Page 43 OK
[LIST] Page 44 OK
[LIST] Page 45 OK
[LIST] Page 46 OK
[LIST] Page 47 OK
[LIST] Page 48 OK
[LIST] Page 49 OK
[LIST] Page 50 OK
[LIST] Page 51 OK
[LIST] Page 52 OK
[LIST] Page 53 OK
[LIST] Page 54 OK
[LIST] Page 55 OK
[LIST] Page 56 OK
[

id, title, slug, url, user_id, contents_short, published_at, updated_at, views_count